In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
import numpy as np
import xarray as xr
import ctdproc as ctd
import matplotlib.pyplot as plt
import warnings
from pathlib import Path

In [84]:
ds = xr.open_dataset("../../data/BLT2/proc/CTD_proc_overturn/nc/blt_ctd_prof_dc_003.nc")

In [85]:
ds.close()

In [86]:
stn = 3

ctd_raw = Path('/Users/hdrake/Documents/data/BLT2/raw/CTD/')
file = f'DY138_{stn:03d}.hex'
hexfile = ctd_raw.joinpath(file)
print(f'Converting hex file for stn {stn}...')
c = ctd.io.CTD(hexfile)
cx = c.to_xarray()
print('Cleaning data...')
datad, datau = ctd.proc.run_all(cx)

Converting hex file for stn 3...


AttributeError: module 'ctdproc.io' has no attribute 'CTD'

In [64]:
print('Depth binning...')
dz = 1
zmin = 15
zmax = np.floor(datad.depth.max().data)

#datad1m = ctd.proc.ctd_bincast(datad, dz, zmin, zmax)
#datau1m = ctd.proc.ctd_bincast(datau, dz, zmin, zmax)

Depth binning...


In [66]:
data = datad.copy()

In [68]:
data

<xarray.Dataset>
Dimensions:  (time: 38144)
Coordinates:
  * time     (time) datetime64[ns] 2021-09-29T02:25:36 ... 2021-09-29T02:52:0...
    depth    (time) float64 2.472 2.444 2.419 2.398 ... 919.2 919.2 919.2 919.1
Data variables: (12/19)
    lon      (time) float64 -10.62 -10.62 -10.62 -10.62 ... -10.62 -10.62 -10.62
    lat      (time) float64 54.73 54.73 54.73 54.73 ... 54.73 54.73 54.73 54.73
    t1       (time) float64 nan nan nan nan nan nan ... nan nan nan nan 7.995
    t2       (time) float64 nan nan nan nan nan nan ... nan nan nan nan 7.992
    c1       (time) float64 nan nan nan nan nan nan ... nan nan nan nan 3.684
    c2       (time) float64 nan nan nan nan nan nan ... nan nan nan nan 3.684
    ...       ...
    CT1      (time) float64 nan nan nan nan nan nan ... nan nan nan nan 7.886
    CT2      (time) float64 nan nan nan nan nan nan ... nan nan nan nan 7.884
    th1      (time) float64 nan nan nan nan nan nan ... nan nan nan nan 7.896
    th2      (time) float64 nan nan nan nan nan nan ... nan nan nan nan 7.893
    sg1      (time) float64 nan nan nan nan nan nan ... nan nan nan nan 27.48
    sg2      (time) float64 nan nan nan nan nan nan ... nan nan nan nan 27.48
Attributes:
    tau1:     0.05439255226189245
    tau2:     0.05472948734981419
    L1:       -0.01868494925101952
    L2:       -0.010448363742329417

In [69]:
dz2 = dz / 2
zbin = np.arange(zmin - dz2, zmax + dz + dz2, dz)
zbinlabel = np.arange(zmin, zmax + dz, dz)

# prepare dataset
tmp = data.swap_dims({"time": "depth"})
tmp = tmp.reset_coords()


In [76]:
# need to bin time separately, not sure why
btime = tmp.time.groupby_bins(
    "depth", bins=zbin, labels=zbinlabel, right=True, include_lowest=True
).mean()
# bin all variables
out = tmp.groupby_bins(
    "depth", bins=zbin, labels=zbinlabel, right=True, include_lowest=True
).mean()


In [80]:
out.coords["time"] = btime

In [29]:
ds['CT1'].plot(y='depth')
plt.ylim(2200, 0)

FileNotFoundError: [Errno 2] No such file or directory: b'/Users/hdrake/Documents/data/DY138/CTD_proc_overturn/nc/blt_ctd_prof_dc_003.nc'

In [27]:
len(1)

TypeError: object of type 'int' has no len()